In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.layers import Dense, Dropout
from tensorflow.keras import layers
from keras.layers import Activation
from keras.models import Sequential

import prettytable
from sklearn import metrics
from math import sqrt

from multiprocessing.pool import ThreadPool


In [2]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
combinations = list()
def get_combinations(arr):
    n = len(arr)
    indices = [0 for i in range(n)]
    while (1):
        res = list()
        for i in range(n):
            res.append(arr[i][indices[i]])
        combinations.append(res)
        next = n - 1
        while (next >= 0 and
              (indices[next] + 1 >= len(arr[next]))):
            next-=1
        if (next < 0):
            return
        indices[next] += 1
        for i in range(next + 1, n):
            indices[i] = 0
u = [25,75,50,100]
b = [25,32,48,64]
e = [25,50,75,100]
arr = [u,b,e]
get_combinations(arr)

In [ ]:
def create_model(X_train,layers=3,units = 32):
    model = Sequential()
    model.add(Dense(units=units,input_dim=(X_train.shape[1]),activation = 'relu'))
    model.add(Dense(units=units,activation = 'relu'))
#     model.add(Dropout(0.2))
#     for i in range(layers):
#         model.add(Dense(units=units,activation = 'relu'))
#         model.add(Dropout(0.2))
    model.add(Dense(units = 1,activation = 'relu'))
    return model

In [ ]:
def backpropogation(X,Y,layers = 3,epochs = 5, batch_size = 32,units=50):

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
    model = create_model(x_train,layers,units)
    model.compile(loss="mse", optimizer="Adam", metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=100, epochs=30, validation_data = (x_test, y_test),shuffle = False,verbose=0)
    y_pred = model.predict(x_test)
    
    rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    c = 0
    for a,b in zip(y_pred,y_test):
        if a * b >= 0:
            c += 1
    direction = c/len(y_test)
    
    myres =  {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse,"rsquared_adj":r2}
    myres.update({"units":units,"epochs":epochs,"batch_size":batch_size,"direction":direction})
    print("done")
    return myres

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        df = pd.read_csv(filepath)
        column = "Next Day Close Price GR"
        df = pre_process_data(df,60)
        (df,column) = dependent_column(df,column)
        X = df[df.columns[:-1]]
        Y = df[column]
        arguments = list()
        for units,batch_size,epochs in combinations:
            data = [X,Y,layers,epochs,batch_size,units]
            arguments.append(data)
        threads = ThreadPool(4)
        result = threads.starmap(backpropogation,arguments)
        resultdf = pd.DataFrame(result)
        resultdf.to_csv(os.path.join(os.getcwd(),str(filename[2:8])+"_bpnn"+".csv"),index=None)

In [4]:
path = os.path.join(os.getcwd(),"Results")

In [20]:
best_bpnn = pd.DataFrame()
for filename in os.listdir(path):
    if filename.endswith("_bpnn.csv") and "best" not in filename:
        df = pd.read_csv(os.path.join(path,filename))
        table = prettytable.PrettyTable()
        table.field_names = df.columns
        table.title = str(filename[:6]) + " bpnn"
        for _,row in df.iterrows():
            row = [round(r,6) for r in row]
            table.add_row(row)
        df["security id"] = str(filename[:6])
        print(table)
        best_bpnn = best_bpnn.append(df.sort_values(by=["direction"],ascending=[False]).head(1),ignore_index=True)
best_bpnn.to_csv(os.path.join(path,"best_bpnn"+".csv"),index=None)

+-----------------------------------------------------------------------------------------------------------------------------+
|                                                         500112 bpnn                                                         |
+-------------------------+---------------------+--------------------+--------------+-------+--------+------------+-----------+
| root_mean_squared_error | mean_absolute_error | mean_squared_error | rsquared_adj | units | epochs | batch_size | direction |
+-------------------------+---------------------+--------------------+--------------+-------+--------+------------+-----------+
|         0.292691        |       0.027469      |      0.085668      |  -0.001022   |  25.0 |  25.0  |    25.0    |    1.0    |
|         0.293204        |       0.028601      |      0.085969      |  -0.004534   |  25.0 |  50.0  |    25.0    |  0.992205 |
|         0.292691        |       0.027469      |      0.085668      |  -0.001022   |  25.0 |  75.0  |  

In [8]:
best_bpnn = pd.read_csv("F:\PracticumProject\StockAnalysisTool\Results\\best_bpnn.csv")
besttable = prettytable.PrettyTable()
cols = ["security id","batch_size","epochs","units"]
best_bpnn = best_bpnn[cols]
besttable.field_names = best_bpnn.columns
besttable.title = "best" + " bpnn"
for _,row in best_bpnn.iterrows():
    row = [round(r,6) if isinstance(r,(int,float)) else r for r in row]
    besttable.add_row(row)
print(besttable)

+-------------------------------------------+
|                 best bpnn                 |
+-------------+------------+--------+-------+
| security id | batch_size | epochs | units |
+-------------+------------+--------+-------+
|    500112   |     25     |   25   |   25  |
|    500180   |     25     |   25   |   25  |
|    500182   |     25     |   25   |   25  |
|    500209   |     25     |   25   |   25  |
|    500325   |     25     |   25   |   25  |
|    500680   |     25     |   25   |   25  |
|    507685   |     25     |   25   |   25  |
|    530965   |     25     |   25   |   25  |
|    532174   |     25     |   25   |   25  |
|    532210   |     25     |   25   |   50  |
|    532540   |     25     |   25   |   50  |
+-------------+------------+--------+-------+
